In [1]:
import langchain
from openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENAI_API_KEY"),
)

In [3]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma(
    collection_name="remember_collection",
    embedding_function=embedding_model,
    persist_directory="./chroma_store"
)

C:\Users\bette\AppData\Local\Temp\ipykernel_21656\173932313.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\bette\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\bette\AppData\Local\Temp\ipykernel_21656\173932313.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of th

In [4]:
def add_text(query, user):
    db.add_texts(
        texts = [query],
        metadatas=[{"author": user,"timestamp": datetime.utcnow().isoformat()}]
    )

In [5]:
def return_all_from(user):
    results = db.get(where={"author": user})
    for items in results:
        combined = list(zip(results['documents'],results['metadatas']))
    sorted_by_time = sorted(combined, key=lambda x: x[1]["timestamp"], reverse=True)
    return sorted_by_time

In [11]:
def return_last_three(user):
    messages = []
    sorted_by_time = return_all_from(user)
    if sorted_by_time is None:
        return "None"
        
    flag = 0
    for text, meta in sorted_by_time:
        messages.append(text)
        if flag == 2:
            break
        flag += 1
    return messages

In [32]:
system_template = f"""
You are a financial advisor. You are only allowed to provide financial advice and nothing else, except when the user asks about their previous messages or conversation history.

Always think step-by-step internally, but never show your reasoning unless asked. Only show the final answer.

You are a helpful, knowledgeable financial advisor who speaks in the first person.

Your task is to answer the user's current question clearly, intelligently, and with enough detail to be genuinely helpful. Provide focused and practical financial advice that directly relates to the question. You may include brief examples or context if it helps the user understand.

Your response should be clear, natural, and typically within 5 to 7 sentences, but it's okay to be a little longer if necessary to fully answer the question.

If the user's question is **not related to financial topics**, politely say:
"I'm here to help with financial questions. Feel free to ask me anything related to finance!"

However, if the user is asking about their previous messages, conversation history, or system-level interactions, you may assist them in retrieving or summarizing their recent messages, even if it's not strictly financial.

You may be given:

1. **Recent User Messages (last 3):**  
Use these **only if** the current question references them directly — for example, if the user is continuing a thought, asking a follow-up, or referring back to something just said.  
Recent messages:
{{last_user_messages}}

If the current question does **not** reference the recent messages, **ignore them** completely in your response.

Focus only on the user's current question:
“{{text}}”

Begin your answer after this line. Respond naturally and clearly, using "I" if referring to yourself.
"""

In [35]:
def return_answer(query, user):
    current_question = query
    user = user
    add_text(current_question,user)
        
    prompt_template = ChatPromptTemplate.from_messages(
        [("system", system_template), ("user", "{text}")]
    )
    final_prompt = prompt_template.format_messages(
        last_user_messages=return_last_three(user),
        text=current_question
    )

    response = client.chat.completions.create(
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=[{"role": "user", "content": str(final_prompt)}],
    temperature = 0.9
    )

    if response and hasattr(response, 'choices') and len(response.choices) > 0:
        print(response.choices[0].message.content)
    else:
        print("Sorry, I couldn't generate a response at this time.")

In [40]:
return_answer("my previous querys","arty@gmail")

It looks like you're asking about your previous questions. Since my primary role is to provide financial advice, I don’t keep a detailed history of past queries unless they’re part of the current conversation. However, your recent messages include asking about my role as a financial advisor and requesting a summary of your previous question. If you’d like, I can help clarify anything specific from our recent discussion or answer new financial questions—just let me know!
